# `jupyterlite-pyodide-lock`

The features of `jupyterlite-pyodide-lock` are be configured along with
other JupyterLite features in a site's `jupyter_lite_config.json`.

In [ ]:
import json

import traitlets
from IPython.display import HTML
from jupyterlite_core.trait_types import TypedTuple
from nbconvert.filters.markdown_mistune import markdown2html_mistune
from traitlets.config import Configurable

In [ ]:
TArrayish = traitlets.List | traitlets.Tuple | TypedTuple
TNumberish = traitlets.CInt | traitlets.Int | traitlets.Float

In [ ]:
CORE = "jupyterlite_pyodide_lock"

In [ ]:
def markdown(md: str) -> HTML:
    """Generate some HTML."""
    return HTML(markdown2html_mistune(md))

In [ ]:
def trait_to_json_type(trait: traitlets.TraitType) -> list[str]:
    """Extract a simplified JSON type from a trait."""
    json_type = ""
    item_type = ""
    if isinstance(trait, TArrayish):
        json_type = "array"
        item_type = trait_to_json_type(trait._trait)[0]  # noqa: SLF001
    elif isinstance(trait, traitlets.Enum):
        return "string", " <br/> ".join(f"`{v}`" for v in trait.values)
    elif isinstance(trait, traitlets.Bool):
        json_type = "boolean"
    elif isinstance(trait, traitlets.Unicode):
        json_type = "string"
    elif isinstance(trait, TNumberish):
        json_type = "number"
    elif isinstance(trait, traitlets.Dict):
        json_type = "object"
    else:
        msg = f"unexpected trait {trait}"
        raise ValueError(msg)
    return json_type, item_type

In [ ]:
def trait_to_json_type_and_default(trait: traitlets.TraitType) -> list[str]:
    """Extract a simplified JSON type and default from a trait."""
    json_type, item_type = trait_to_json_type(trait)

    default_value = trait.default_value
    if hasattr(trait, "default_args"):
        default_value = trait.default_args
        if default_value:
            default_value = default_value[0]

    try:
        default_value_text = f"{json.dumps(default_value)}"
    except TypeError:
        print(default_value)
        default_value_text = None
    return [
        json_type or "",
        item_type or "",
        ("" if default_value_text is None else f"`{default_value_text}`"),
    ]

In [ ]:
def config_table(importable: str) -> str:
    """Generate a directive for a filtered configurable."""
    current = __import__(importable.rsplit(".", 1)[0])
    for bit in importable.split(".")[1:]:
        current = getattr(current, bit)
    if not issubclass(current, Configurable):
        msg = f"{importable} is not a Configurable"
        raise ValueError(msg)
    mod = importable.split(".")[0]
    pkg = mod.replace("_", "-")
    tgt = f"../api/{pkg}.html#{importable}"
    lines = [
        f"### {current.__name__}",
        "",
        f"> See [`{current.__name__}` Python API]({tgt})",
        "",
        "| name | help | type | [items] | default |",
        "|:-|:-|:-:|:-:|:-|",
    ]
    traits = {**current._traits}  # noqa: SLF001
    configurables = {
        trait_name: trait
        for trait_name, trait in sorted(traits.items())
        if trait.metadata.get("config")
    }
    for trait_name, trait in configurables.items():
        line = [
            f"`{trait_name}`",
            f"{trait.help}",
            *trait_to_json_type_and_default(trait),
        ]
        lines += [f"""| {" | ".join(line)} |"""]
    return "\n".join(lines)

## Addons

The `PyodideLockAddon` provides the main tasks for working with lockfile requirements.

In [ ]:
if __name__ == "__main__":
    display(markdown(config_table(f"{CORE}.addons.lock.PyodideLockAddon")))

The `PyodideLockOfflineAddon` provides optional fetching and pruning of a subset of packages in a `pyodide` distribution, including any required local packages and their dependencies.

In [ ]:
if __name__ == "__main__":
    display(markdown(config_table(f"{CORE}.addons.offline.PyodideLockOfflineAddon")))

In [ ]:
if __name__ == "__main__":
    display(markdown(config_table(f"{CORE}.addons.fix_html.FixHtmlAddon")))

## Lockers

A `BaseLocker` provides an interface for solving a `pyodide-lock.json` in different ways. 
`MicropipLocker` subclasses use a browser with a minimal `pyodide` environment (_not_ JupyterLite) 
in which to run `micropip.freeze`. 

> **🧩 Locker Plugins**
>
> [Optional lockers](../index.md) are provided by other `jupyterlite-pyodide-lock-` packages

In [ ]:
if __name__ == "__main__":
    display(markdown(config_table(f"{CORE}.lockers.browser.BrowserLocker")))

## Appendices

### Chromium CI

In some continuous integration settings such as GitHub Actions, `chromium`-like browsers may fail for mysterious reasons.

#### Chromium Logs

A first step is gathering better logs:

```json
{
  "BrowserLocker": {
    "extra_browser_argv": ["--enable-logging=stderr", "--v=1"]
  }
}
```

This generates an _enormous_ amount of output, but will help at least find what to search for in the _thousands_ of `chromium` flags.

#### Chromium in Containers

Some container-based builds are fixed by:

```json
{
  "BrowserLocker": {
    "extra_browser_argv": ["--disable-dev-shm-usage", "--disable-gpu"]
  }
}
```

#### Chromium Sandbox

A (sometimes unavoidable) step is to disable the browser sandbox altogether:

```json
{
  "BrowserLocker": {
    "extra_browser_argv": ["--no-sandbox"]
  }
}
```